<a href="https://colab.research.google.com/github/jeaneigsi/cookbook/blob/main/LlamaCpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
!pip install llama-cpp-python

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 987.2 kB/s eta 0:00:00


In [8]:
!pip3 install hf_transfer


In [9]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False


downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf to /root/.cache/huggingface/hub/tmpo57yloz6
mistral-7b-instruct-v0.2.Q5_K_M.gguf: 100% 5.13G/5.13G [00:41<00:00, 123MB/s]
./mistral-7b-instruct-v0.2.Q5_K_M.gguf


In [10]:
!pip3 install huggingface-hub

In [11]:
prompt= """
### System:
You are an AI assistant that follows instructions exceptionally well. Be as helpful as possible.
### User:
List the top 10 APIs
### Assistant:
"""

In [ ]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf",  # Download the model file first
  n_ctx=20000  # The max sequence length to use - note that longer sequence lengths require much more resource
)

In [30]:
output = llm(
   "Q: Who is the president of USA? A: ",
   max_tokens=20000,
   stop=["Q:", "\n"],
   echo=True
)
print(output)


llama_print_timings:        load time =    7311.48 ms
llama_print_timings:      sample time =      47.66 ms /    79 runs   (    0.60 ms per token,  1657.61 tokens per second)
llama_print_timings: prompt eval time =    7310.90 ms /    13 tokens (  562.38 ms per token,     1.78 tokens per second)
llama_print_timings:        eval time =   66837.78 ms /    78 runs   (  856.89 ms per token,     1.17 tokens per second)
llama_print_timings:       total time =   74520.83 ms /    91 tokens


{'id': 'cmpl-fbae687c-1b1c-431e-b0e3-3eb126de46ae', 'object': 'text_completion', 'created': 1713659576, 'model': '/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf', 'choices': [{'text': 'Q: Who is the president of USA? A:  Joe Biden is the President of the United States as of January 20, 2021. He assumed office following his victory in the presidential elections held on November 3, 2020. Prior to Biden, Donald Trump served as the President from January 20, 2017, to January 20, 2021.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 78, 'total_tokens': 91}}


In [28]:
output=output['choices'][0]['text']
print(output)

Q: Who is the president of USA? A:  As of my knowledge up to 2021, the President of the United States is Joe Biden. He assumed office on January 20, 


In [ ]:
!pip install langchain_community langchain_core langchain

In [32]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [33]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate.from_template(template)

In [34]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [36]:
question = """
Question: A rap battle between Stephen Colbert and John Oliver
"""
llm.invoke(question)

Answer: (Verse 1 - Stephen Colbert)
Yo, I'm the Colbert Report,
I got the wit, I got the report,
Now John Oliver, you think you're so slick,
But when it comes to satire, Colbert's got theicks!

(Chorus - Stephen Colbert)
So step up, John Oliver, and try to keep up with me,
But when it comes to comedy, Colbert's got the key!

(Verse 2 - John Oliver)
Stephen Colbert, you may have the report,
But when it comes to taking on the powerful, I've got the sort!

(Chorus - John Oliver)
So come on, Stephen Colbert, and try to keep up with me,
But when it comes to holding the powerful accountable, Oliver's got the key!

(Outro - Both)
So there you have it, a rap battle for the ages,
Stephen Colbert and John Oliver, we'll let you engage,
But no matter who comes out on top, one thing's for sure, satire's alive and well, and Colbert and Oliver, they're just doing their best!

(End of Rap Battle)


llama_print_timings:        load time =    8770.28 ms
llama_print_timings:      sample time =     184.92 ms /   282 runs   (    0.66 ms per token,  1525.01 tokens per second)
llama_print_timings: prompt eval time =   15907.55 ms /    16 tokens (  994.22 ms per token,     1.01 tokens per second)
llama_print_timings:        eval time =  256126.45 ms /   281 runs   (  911.48 ms per token,     1.10 tokens per second)
llama_print_timings:       total time =  274147.36 ms /   297 tokens


"Answer: (Verse 1 - Stephen Colbert)\nYo, I'm the Colbert Report,\nI got the wit, I got the report,\nNow John Oliver, you think you're so slick,\nBut when it comes to satire, Colbert's got theicks!\n\n(Chorus - Stephen Colbert)\nSo step up, John Oliver, and try to keep up with me,\nBut when it comes to comedy, Colbert's got the key!\n\n(Verse 2 - John Oliver)\nStephen Colbert, you may have the report,\nBut when it comes to taking on the powerful, I've got the sort!\n\n(Chorus - John Oliver)\nSo come on, Stephen Colbert, and try to keep up with me,\nBut when it comes to holding the powerful accountable, Oliver's got the key!\n\n(Outro - Both)\nSo there you have it, a rap battle for the ages,\nStephen Colbert and John Oliver, we'll let you engage,\nBut no matter who comes out on top, one thing's for sure, satire's alive and well, and Colbert and Oliver, they're just doing their best!\n\n(End of Rap Battle)"

In [37]:
llm_chain = prompt | llm

In [38]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.invoke({"question": question})

Llama.generate: prefix-match hit




First, let's find the year Justin Bieber was born. Justin Bieber was born on March 1, 1994.

Next, let's find which NFL team won the Super Bowl in the year Justin Bieber was born.

The Super Bowl was first played on January 15, 1967. Therefore, the Super Bowl wasn't played in the same year as Justin Bieber's birth.

Therefore, there is no NFL team that won the Super Bowl in the year Justin Bieber was born because the Super Bowl wasn't played in that year.


llama_print_timings:        load time =    8770.28 ms
llama_print_timings:      sample time =      88.11 ms /   138 runs   (    0.64 ms per token,  1566.19 tokens per second)
llama_print_timings: prompt eval time =   31100.06 ms /    45 tokens (  691.11 ms per token,     1.45 tokens per second)
llama_print_timings:        eval time =  123643.38 ms /   137 runs   (  902.51 ms per token,     1.11 tokens per second)
llama_print_timings:       total time =  155726.70 ms /   182 tokens


"\n\nFirst, let's find the year Justin Bieber was born. Justin Bieber was born on March 1, 1994.\n\nNext, let's find which NFL team won the Super Bowl in the year Justin Bieber was born.\n\nThe Super Bowl was first played on January 15, 1967. Therefore, the Super Bowl wasn't played in the same year as Justin Bieber's birth.\n\nTherefore, there is no NFL team that won the Super Bowl in the year Justin Bieber was born because the Super Bowl wasn't played in that year."